# Проект "Как изменилась тематика фильмов за последние 20 лет"    
## Парсинг.
*Источник данных - неофициальный API Кинопоиска https://kinopoisk.dev/*  
*Сравниваемые периоды: 2020-2025 гг. и 2000-2005 гг.*

*API позволяет загружать по 1 странице, состоящей из списка 250 фильмов с описанием.*

Для анализа отберу фильмы по таким фильтрам:
- по количеству оценок в порядке убывания, чтобы отобрать наиболее популярные фильмы;
- поставлю фильтр, что это должен быть именно фильм, а не мультфильм, сериал и т.п.;
- отмечу, что описание фильма != 'Null';
- а также отмечу период 2020-2025гг, а для второго корпуса 2000-2005гг.

*Просмотрев результаты поиска, я решила взять первые 50 страниц поисковой выдачи.
Должно получиться около 12 500 фильмов в каждом корпусе.*

In [ ]:
import requests
import re

In [ ]:
# API позволяет загружать по 1 странице, состоящей из списка 250 фильмов с описанием.
# изначально я получила такой код для загрузки первой страницы поисковой выдачи:

url = "https://api.kinopoisk.dev/v1.4/movie?page=1&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2020-2025"

headers = {
    "accept": "application/json",
    "X-API-KEY": "AW8KS88-T27M50N-P61YKCP-1GSXSJX"
}

response = requests.get(url, headers=headers)

response = response.text

In [ ]:
# посмотрим, в каком виде мы получаем страницы:
# response

In [ ]:
# для проекта возьмём первые 50 страниц поисковой выдачи.
# для этого сгенерируем ссылки на них в цикле и сохраним в links_1_period и links_2_period:
links_1_period = []
links_2_period = []

for i in range(1, 51):
    links_1_period.append(f"https://api.kinopoisk.dev/v1.4/movie?page={str(i)}&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2020-2025")
    links_2_period.append(f"https://api.kinopoisk.dev/v1.4/movie?page={str(i)}&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005")

links_2_period[:3]

['https://api.kinopoisk.dev/v1.4/movie?page=1&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005',
 'https://api.kinopoisk.dev/v1.4/movie?page=2&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005',
 'https://api.kinopoisk.dev/v1.4/movie?page=3&limit=250&notNullFields=description&sortField=votes.kp&sortType=-1&type=movie&year=2000-2005']

In [ ]:
# а теперь загрузим все эти странички:
def load_pages(links):
    pages = []

    for link in links:
        page = requests.get(link, headers=headers).text
        pages.append(page)
    return(pages)

In [ ]:
pages_1 = load_pages(links_1_period)
pages_2 = load_pages(links_2_period)
pages_2[-1]

In [ ]:
# достанем описание к каждому фильму и сохраним его в переменную с помощью функции:
def collect_descriptions(pages_list):
    all_descriptions = []

    for page in pages_list:
        page = re.sub(r'(\xa0|\\n|\\r)', ' ', page)  # сразу заменим неразрывные пробелы и переносы строк на пробел
        descriptions = re.findall(r'"description":"([А-ЯЁA-Za-zа-яё].+?)","', page)  # среди описаний всё-таки есть пустые, не отмеченные как 'Null'
        # поэтому в регулярном выражении укажем, что нас интересуют описания, начинающиеся с буквы.

        all_descriptions.append(descriptions)
        return(all_descriptions)

In [ ]:
all_descriptions_1 = collect_descriptions(pages_1)
all_descriptions_2 = collect_descriptions(pages_2)

In [ ]:
# оценим объём получившегося корпуса:
def mesure_volume(all_descriptions):
    films_base = 0  # здесь посчитаем количество фильмов в коллекции
    corpus_volume = 0  # а здесь - количество токенов

    for descriptions_list in all_descriptions:
        #print(len(i))  # можно посмотреть, сколько фильмов с каждой странички вошли в корпус
        films_base += len(descriptions_list)
        for description in descriptions_list:
            corpus_volume += len(description.split())
    return(films_base, corpus_volume)

In [ ]:
mesure_volume(all_descriptions_1), mesure_volume(all_descriptions_2)
# итак, получилось 2 корпуса: первый объёмом 480 266 токенов и второй объёмом 719 381 токен. В обоих корпусах около 12 000 фильмов.

((11837, 480266), (11981, 719381))

In [ ]:
# прербразуем для удобства получившиеся списки списков в плоские списки:
def create_flat_list(list):
    flat_list = sum(list, [])
    return(flat_list)

In [ ]:
all_descriptions_1 = create_flat_list(all_descriptions_1)
all_descriptions_2 = create_flat_list(all_descriptions_2)

In [ ]:
# проверим, что получилось:
# all_descriptions_1[:5]

In [ ]:
# соберём датафрейм из полученных данных и далее сохраним в файл:
import pandas as pd
df_1 = pd.DataFrame(all_descriptions_1, columns=['Описание фильма'])
df_1.head()

,Описание фильма
0,Грузовики лос-анджелесской инкассаторской комп...
1,"Гриша, бывший мажор, побывавший холопом и став..."
2,"Если ты идешь на рыбалку — будь готов к тому, ..."
3,"Афганистан, март 2018 года. Во время спецопера..."
4,История о путешествии взрослого героя к своему...


In [ ]:
df_2 = pd.DataFrame(all_descriptions_2, columns=['Описание фильма'])
df_2.head()

,Описание фильма
0,"Участвуя в программе на телевидении, Данила Ба..."
1,Жизнь десятилетнего Гарри Поттера нельзя назва...
2,"Жизнь харизматичного авантюриста, капитана Дже..."
3,"Фрэнк Эбегнейл успел поработать врачом, адвока..."
4,Римская империя. Бесстрашного и благородного г...


In [ ]:
# сохраним результаты
df_1.to_excel('Descriptions_1.xlsx')
df_2.to_excel('Descriptions_2.xlsx')